In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer
from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
from sparkstudy.ml.transformers import DemoTransformer
%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


最简单的hello world例子。也没有什么太特别的东西。
1. pipeline和scipy的有一点不一样。这里的fit和transformer是强行分开的。

In [5]:
sessionFactory = DemoSQLSessionFactory(name="pipeline exampe")
spark = sessionFactory.build_session()
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [6]:
pipeline = Pipeline(stages=[tokenizer])
model = pipeline.fit(training)
model.transform(training).show()

+---+----------------+-----+--------------------+
| id|            text|label|               words|
+---+----------------+-----+--------------------+
|  0| a b c d e spark|  1.0|[a, b, c, d, e, s...|
|  1|             b d|  0.0|              [b, d]|
|  2|     spark f g h|  1.0|    [spark, f, g, h]|
|  3|hadoop mapreduce|  0.0| [hadoop, mapreduce]|
+---+----------------+-----+--------------------+



主要看看有没有任何嵌套。

In [8]:
tokenize_1 = Tokenizer(inputCol="text", outputCol="words_1")
pipeline_1 = Pipeline(stages=[tokenize_1,pipeline])
pipeline_1.fit(training).transform(training).show()

+---+----------------+-----+--------------------+--------------------+
| id|            text|label|             words_1|               words|
+---+----------------+-----+--------------------+--------------------+
|  0| a b c d e spark|  1.0|[a, b, c, d, e, s...|[a, b, c, d, e, s...|
|  1|             b d|  0.0|              [b, d]|              [b, d]|
|  2|     spark f g h|  1.0|    [spark, f, g, h]|    [spark, f, g, h]|
|  3|hadoop mapreduce|  0.0| [hadoop, mapreduce]| [hadoop, mapreduce]|
+---+----------------+-----+--------------------+--------------------+



自定义的transformer
1. 感觉相对来说，还是比较简单的。就是处理transformer的一个方法。而且传入的是全集

In [17]:
custom_transformer = DemoTransformer()
custom_pipeline = Pipeline(stages=[custom_transformer])
custom_pipeline.fit(training).transform(training).show()

+---+----------------+-----+---+
| id|            text|label|new|
+---+----------------+-----+---+
|  0| a b c d e spark|  1.0| 15|
|  1|             b d|  0.0|  3|
|  2|     spark f g h|  1.0| 11|
|  3|hadoop mapreduce|  0.0| 16|
+---+----------------+-----+---+

